In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import ngrams
from itertools import chain
from wordcloud import WordCloud
import re
import pickle
from sklearn.metrics import confusion_matrix
from scipy.optimize import least_squares

In [2]:
fulltrainingSet = pd.read_pickle('modifiedtrainingSet.pickle')
test_index = pd.read_csv("test.csv")['Id'].tolist()

In [3]:
fulltrainingSet['reviews'] = fulltrainingSet['Summary'] + fulltrainingSet['Text']
print(fulltrainingSet['reviews'])

0          good version of a classicThis is a charming ve...
1          Good but not as movingIt was good but not as e...
2          Winkler's Performance was ok at best!Don't get...
3          It's an enjoyable twist on the classic storyHe...
4          Best Scrooge yetThis is one of the best Scroog...
                                 ...                        
1697528    Way to Expensive!! WB = GREEDwow $269.99 for t...
1697529    HOLY BAT-BOXSET, BATMAN... I never thought thi...
1697530    prayers have been answered because batman 60s ...
1697531    can't Wait!I've been a fan of the series since...
1697532    The Price is Insane? People Really Need to Wak...
Name: reviews, Length: 1697533, dtype: object


In [4]:
trainingSet = fulltrainingSet.dropna()
train_index = trainingSet.index.tolist()

In [5]:
Y = np.array(trainingSet['Score'])
X = trainingSet[['ProductId','UserId','reviews']]
sub_x = trainingSet['reviews']

In [6]:
full_sub_x = fulltrainingSet['reviews']

In [7]:
c = CountVectorizer(stop_words = 'english')


In [8]:
c = CountVectorizer(stop_words = 'english', min_df = 5000, max_df = 1000000)
X = c.fit_transform(full_sub_x)

In [9]:
print(X.shape)

(1697533, 3451)


In [10]:
train_x = X[train_index]
train_y = fulltrainingSet['Score'].iloc[train_index].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

In [11]:
clf = LogisticRegression(solver = "saga", max_iter = 200)
clf_model = clf.fit(X_train,y_train)

/Users/lw/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [12]:
prediction = clf.predict(X[test_index])
print(prediction.shape)

(300000,)


In [25]:
prob_predict = np.around(np.matmul(clf.predict_proba(X[test_index]),np.array([1,2,3,4,5])),0)
print(prob_predict)

[5. 4. 4. ... 4. 5. 4.]


In [14]:
col_result = np.matmul(clf.predict_proba(X),np.array([1,2,3,4,5]))

In [28]:
print(col_result)

[4.40565317 4.24312802 4.07796109 ... 4.83302686 4.21819137 3.82620512]


In [22]:
fulltrainingSet['result'] = col_result

In [24]:
fulltrainingSet.to_pickle("result.pickle")